In [2]:
import pandas as pd
import numpy as np
import re
import jieba

In [3]:
train_tcn = pd.read_csv('./datasets/train_tcn.csv')
dev_tcn = pd.read_csv('./datasets/dev_tcn.csv')
test_tcn = pd.read_csv('./datasets/test_tcn.csv')

In [4]:
train_tcn.head()

,product_title,category
0,Gucci Gucci Guilty Pour Femme Stud Edition 罪愛女...,Health & Beauty
1,（二手）PS4 GTA 5 俠盜獵車手5 Grand Theif Auto V繁體 中文版,Game Kingdom
2,百獸卡,Life & Entertainment
3,nac nac活氧全效柔衣素,Mother & Baby
4,#Nike耐吉官方F.C. 男子足球長褲新款標準型 拒水 拉鏈褲腳\nCD0557,Men's Apparel


In [5]:
dev_tcn.head()

,text,split
0,OPPO A75 A75s A73 手机壳 软壳 挂绳壳 大眼兔硅胶壳,private
1,SOFT 99 鍍膜車蠟(強力撥水型),private
2,低糖芒果乾 250g 臻御行,private
3,＊小徑文化＊日本進口ROUND TOP space craft - diamond (SC-...,private
4,Hello Kitty 凱蒂貓 KITTY 涼鞋 童鞋 白/紅色 小童 no739,private


In [6]:
test_tcn.head()

,text,split
0,【PolarStar】美麗諾羊毛保暖襪『淺灰』P18634,public
1,甜蜜水晶~天然水晶五行珠手鍊10mm手鍊,public
2,粉晶六角柱純銀項鍊,public
3,3M SCOTCH VHB 超強力雙面膠-戶外專用 V1808,public
4,燈專屬優惠 *4盒,public


In [7]:
category_tcn = train_tcn.category.unique()

print(category_tcn)
print(len(category_tcn))

['Health & Beauty' 'Game Kingdom' 'Life & Entertainment' 'Mother & Baby'
 "Men's Apparel" "Women's Apparel" 'Women Bags' 'Women Shoes'
 'Home & Living' 'Everything Else' 'Hardware & 3C'
 "Men's Bags& Accessories" 'Sports & Outdoors' 'Helpbuy' 'Men Shoes'
 'Books' 'Motors' 'Home Electronic' 'Food & Beverages' 'Mobile & Gadgets'
 'Women Accessories' 'Pets' 'Tickets & Services' nan]
24


In [8]:
train_tcn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 2 columns):
product_title    499999 non-null object
category         499982 non-null object
dtypes: object(2)
memory usage: 7.6+ MB


## Data Preprocessing

1. punctuation removal 
2. non-chinese word removal
3. numbers removal 
4. emoji/special characters removal
5. tokenization

In [9]:
# remove punctuation, non chinese words, special characters
def non_chinese_removal(line):   
    p1 = re.compile(r'''~!@#$%^&*()_\-+=<>?:"{}|,.\/;'\[]·~！@#￥%……&*（）——\-+={}|「『《》？：“”【】、；‘'，。、''')     
    p2 = re.compile(r'[^\u4e00-\u9fa5]') 
                        
    line = p1.sub(r'',str(line))
    line = p2.sub(r'',str(line))
        
    return line

In [10]:
train_tcn['product_title_clean'] = train_tcn['product_title'].apply(non_chinese_removal)
dev_tcn['product_title_clean'] = dev_tcn['text'].apply(non_chinese_removal)
test_tcn['product_title_clean'] = test_tcn['text'].apply(non_chinese_removal)

In [11]:
train_tcn['product_title_clean'][10]

'居家大推款記憶棉枕護勁頸椎枕修復脊椎治頸椎專用枕頭女孕婦失眠保健理療枕'

In [12]:
def tokenize_chinese(title):
    wordlist = jieba.cut(title) # there are 3 tokenization mode for jieba, after testing, precision mode is better for our case 
    wl_space_split = " ".join(wordlist)
    return wl_space_split

In [13]:
train_tcn['product_title_tokenized'] = train_tcn['product_title_clean'].apply(tokenize_chinese)
dev_tcn['product_title_processed'] = dev_tcn['product_title_clean'].apply(tokenize_chinese)
test_tcn['product_title_processed'] = test_tcn['product_title_clean'].apply(tokenize_chinese)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/83/63jmmq312vsf41phtw7ph8400000gn/T/jieba.cache
Loading model cost 1.332 seconds.
Prefix dict has been built succesfully.


In [14]:
train_tcn['product_title_tokenized'][10]

'居家 大推 款 記憶 棉枕 護勁 頸椎 枕 修復 脊椎 治 頸椎 專用 枕頭 女 孕婦 失眠 保健 理療 枕'

In [15]:
dev_tcn['product_title_processed']

0        手 机 壳 软 壳 挂 绳 壳 大眼 兔 硅 胶 壳
1                      鍍膜 車蠟 強力 撥水型
2                       低糖 芒果乾 臻 御行
3                       小徑 文化 日本 進口
4                 凱蒂貓 涼鞋 童鞋 白 紅色 小童
5                     雷鳥 日式 木紋 桌 墊片
6                             狂砂 小子
7                    巨蟒 泰坦 系列 固態 硬碟
8                  單肩 斜 背包 側背包 肩 背包
9                  大雪山 農場 山 苦瓜 茶包 盒
10                 高露 潔 雙效 潔淨 牙刷 單支
11                               絲緞
12                  芮菲客 米蘭 玫瑰金 鋼珠 筆
13            海底 總動員 玻璃 磁鐵 多莉 款 野獸國
14                       甘草 瓜子 南瓜 子
15                           行車 記錄器
16                       伊士 達分 濾網 頭
17              聚點 方形 方塊 灰藍 連帽 帽男 款
18              流蘇 立體 縫線 莫卡 辛鞋 藍 黑色
19            新品 時尚 可愛 寶寶 繡花 連衣裙 洋裝
20     客制化 泡泡 气囊气 垫 手 机 支架 捕 梦 网 羽毛
21                   機器人 印花 尼龍 手 拿包
22               伊 玛 公升 不 锈 钢 快 煮 壶
23                                送
24         日系 羊毛 混紡 保暖 堆 堆帽 貝雷帽 毛線帽
25                日本 芝研 植萃 皇后 化妝水 款
26     德爾 斐 神諭 繁體 中文版 高雄 龐奇 桌遊 新天鵝堡
27                 印尼 拉 茶包 可

In [16]:
test_tcn['product_title_processed'][:5]

0           美麗 諾 羊毛 保暖 襪 淺灰
1    甜蜜 水晶 天然 水晶 五行 珠 手鍊 手鍊
2             粉晶 六 角柱 純銀 項鍊
3           超 強力 雙面 膠 戶外 專用
4                 燈 專屬 優惠 盒
Name: product_title_processed, dtype: object

In [17]:
train_tcn.to_csv('cleaned_train_tcn.csv', index = False)
dev_tcn.to_csv('cleaned_dev_tcn.csv', index = False)
test_tcn.to_csv('cleaned_test_tcn.csv', index = False)

### Avoid Stopword Removal:

1. Machine Translation

2. Language Modeling

3. Text Summarization

4. Question-Answering problems